# Init

In [ ]:
# init
from importlib import reload
from pathlib import Path
import os
import sys
import pandas as pd

sys.path.append('..')
sys.path.append("C:/Users/gonta/D/software developer/packages")
import toolsGeneral.main as tgm
import toolsPandas.helpers as tph

def reloadPckg():
    reload(tgm)
    reload(tph)

# Import

In [1]:
# import
dataDir = Path(os.path.join(os.getcwd(), '..', 'data/normalized'))
rawFiles = [f for f in dataDir.rglob('*.pkl')]
print(len(rawFiles))

df_by_cntr = {
    file.parent.name:tgm.load(str(file))
    for file in rawFiles
}
print(len(df_by_cntr))

64
64


In [2]:
df_by_cntr['Greece']

,type,id,tags.admin_level,tags.parent_id,tags.name,tags.name:us,tags.ISO3166-1,tags.ISO3166-2,tags.is_in:country,tags.ref:nuts,tags.ref:nuts:2,tags.ref:nuts:3
0,relation,192307,2,NaN,Ελλάς,NaN,GR,NaN,NaN,EL,NaN,NaN
1,relation,223136,4,NaN,Muğla,NaN,NaN,TR-48,NaN,TR323,NaN,TR323
2,relation,4458374,4,NaN,Αποκεντρωμένη Διοίκηση Ηπείρου - Δυτικής Μακεδ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,relation,4477553,4,NaN,Αποκεντρωμένη Διοίκηση Αττικής,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,relation,4495677,4,NaN,Αποκεντρωμένη Διοίκηση Θεσσαλίας - Στερεάς Ελλ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
584,rel,13829626,8,1827294,Balıbey Mahallesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,rel,13829627,8,1827294,Camikebir Mahallesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,rel,13829628,8,1827294,Hacırüstem Mahallesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,rel,13830796,8,1827377,Ulucami Mahallesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apply fixes

## 1. osm basic fix

In [13]:
osm_basic_test = tgm.load(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'))
len(osm_basic_test)

64

In [14]:
tph.df_peek(tph.to_df(osm_basic_test).T)

,missing.name,leak
Albania,[],[]
Algeria,[],[]
Andorra,[],[]
Australia,[],[]
Austria,[],[52345]
Bangladesh,[14326679],"[3824588, 3825003, 3859335, 3921211, 3921222, 3921298, 3921322, 7318343, 9455570]"
Belarus,[],"[59162, 59189, 59195, 59275, 59506, 59752]"
Belgium,[],"[4217435, 7394, 7400, 7411, 7427, 7463, 7400]"
Bhutan,[],[]
Brazil,[],"[54882, 57963, 185579, 242620, 298204, 301079, 301464, 302635, 302819, 303702, 303781, 303940, 315173, 325866, 326266, 331463, 332476, 332924, 333597, 334051, 334443, 336819, 362413, 405935, 421151, 4059665, 298424, 298446]"


In [15]:
to_delete = []
for k,v in osm_basic_test.items():
    to_delete.extend(v['missing.name'])
    to_delete.extend(v['leak'])
len(to_delete)

166

In [16]:
# delete missing names and leaked elements
df_by_cntr_fixed = {}
for k,df in df_by_cntr.items():
    df_by_cntr_fixed[k] = df[~df['id'].isin(to_delete)].copy()

In [7]:
list(map(len,[df_by_cntr['Bangladesh'],df_by_cntr_fixed['Bangladesh']]))

[453, 443]

## 2. osm duplicates fix

In [18]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))

osm_dup_test = {
    file.parent.name:tgm.load(str(file))
    for file in data_dir.rglob('*/*.pkl')
}

In [19]:
to_delete_dup = []
for k,v in osm_dup_test.items():
    to_delete_dup.extend(v['duplicate.delete_elems'])
len(to_delete_dup)

158

## * apply

In [22]:
# delete duplicate elements
for k,df in df_by_cntr_fixed.items():
    df_by_cntr_fixed[k] = df[~df[['id','tags.parent_id']].apply(tuple, axis=1).isin(to_delete_dup)].copy()

# 3. Save!

In [25]:
for k,df in df_by_cntr_fixed.items():
    path = os.path.join(os.getcwd(),f'../data/fixed/{k}/{k}_fixed.pkl')
    tgm.dump(path, df)

# Review

In [26]:
dir = Path(os.path.join(os.getcwd(), '..', 'data/fixed'))
df_by_cntr_fixed = {
    file.parent.name:tgm.load(str(file))
    for file in [f for f in dir.rglob('*/*.pkl')]
}
len(df_by_cntr_fixed)

64

In [27]:
all_rows = pd.concat([df for df in df_by_cntr_fixed.values()], ignore_index=True)
len(all_rows)

118403

In [28]:
all_rows['tags.admin_level'].value_counts()

tags.admin_level
8    99107
6    18342
4      890
2       64
Name: count, dtype: int64

In [29]:
def resume_divisions(rls : dict):
    resume = {}
    for k,df in rls.items():
        add_1 = df[df['tags.admin_level'] == '4']
        add_2 = df[df['tags.admin_level'] == '6']
        add_3 = df[df['tags.admin_level'] == '8']
        resume[k] = {'add_1': add_1, 'add_2': add_2, 'add_3': add_3}
    
    return pd.DataFrame(resume)

In [30]:
resume = resume_divisions(df_by_cntr_fixed)
len(resume)

3

In [31]:
tph.df_peek(resume.T.map(len))

,add_1,add_2,add_3
Albania,3,12,374
Algeria,58,547,1541
Andorra,0,0,0
Australia,15,600,0
Austria,8,93,2075
Bangladesh,0,392,48
Belarus,1,128,1290
Belgium,3,10,4327
Bhutan,20,205,964
Brazil,3,44,711
